<a href="https://colab.research.google.com/github/Ashi743/my_projects-/blob/main/shortest_path_using_Dijkstra's_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 2.4 MB/s eta 0:00:00


In [3]:
!pip install networkx

In [6]:
import folium
import osmnx as ox
import networkx as nx
import ipyleaflet
from ipyleaflet import Map, Marker, Polyline
from IPython.display import display

In [12]:
origin_coordinates= [28.5439, 77.3331]    #amity
destination_coordinates =[28.4595, 77.0266]     #

SG= ox.graph_from_place('Delhi, India', network_type='drive') # Using the correct package name
origin_node= ox.distance.nearest_nodes(SG, origin_coordinates[1], origin_coordinates[0])
destination_node= ox.distance.nearest_nodes(SG, destination_coordinates[1], destination_coordinates[0])

shortest_path= nx.shortest_path(SG, origin_node, destination_node, weight='length')

path_coordinates= [(SG.nodes[node]['y'], SG.nodes[node]['x']) for node in shortest_path]

m= Map(center= [( origin_coordinates[0] + destination_coordinates[0]) /2 , ( origin_coordinates[1] + destination_coordinates[1]) /2], zoom=10)
origin_marker= Marker(location= origin_coordinates, draggable=False, title='amity')
destination_marker= Marker(location= destination_coordinates, draggable=False, title='shimla')
m.add_layer(origin_marker)
m.add_layer(destination_marker)
polyline= Polyline(locations= path_coordinates, color='blue', weight=2, fill= False)
m.add_layer(polyline)
display(m)

Map(center=[28.5017, 77.17985], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

Finding shortest path with dijkstra

In [7]:
import heapq
import folium
from folium.plugins import HeatMap


In [8]:
# geo coord
tattapani= [31.2488, 77.0895]
shimla= [31.1050, 77.1640]
waknaghat= [31.0079, 77.0881]
solan=  [30.9084, 77.0999]
chandigarh= [30.7333, 76.7794]

In [11]:
graph = {
    "tattapani": [('shimla', 54.6), ('waknaghat', 79.3), ('solan', 96.6), ('chandigarh', 130)],
    "shimla": [('tattapani', 54.6), ('waknaghat', 24.7), ('solan', 42), ('chandigarh', 108.5)],
    "waknaghat": [('tattapani', 79.3), ('shimla', 24.7), ('solan', 23.3), ('chandigarh', 92.2)],
    "solan": [('tattapani', 96.6), ('shimla', 42), ('waknaghat', 23.3), ('chandigarh', 68.4)],
    "chandigarh": [('tattapani', 130), ('shimla', 108.5), ('waknaghat', 92.2), ('solan', 68.4)]
}


In [14]:
def find_shortest_path(graph, nodes):
    import itertools

    # Generate all permutations of nodes
    all_paths = itertools.permutations(nodes)
    shortest_path = None
    shortest_path_length = float('inf')

    for path in all_paths:
        # Check if the path starts at 'tattapani' and ends at 'chandigarh'
        if path[0] != 'tattapani' or path[-1] != 'chandigarh':
            continue

        # Calculate the path length
        path_length = 0
        valid_path = True

        for i in range(len(path) - 1):
            current_node = path[i]
            next_node = path[i + 1]

            # Search for the edge in the adjacency list
            found_edge = False
            for neighbor, weight in graph[current_node]:
                if neighbor == next_node:
                    path_length += weight
                    found_edge = True
                    break

            # If no edge exists between current_node and next_node, path is invalid
            if not found_edge:
                valid_path = False
                break

        # Update the shortest path if a valid path is found
        if valid_path and path_length < shortest_path_length:
            shortest_path_length = path_length
            shortest_path = path

    return shortest_path, shortest_path_length


In [15]:
nodes_to_visit= ['tattapani', 'shimla', 'waknaghat', 'solan', 'chandigarh']
shortest_path, shortest_path_length= find_shortest_path(graph, nodes_to_visit)
print("Shortest Path:", shortest_path)
print("Shortest Path Length:", shortest_path_length)

Shortest Path: ('tattapani', 'shimla', 'waknaghat', 'solan', 'chandigarh')
Shortest Path Length: 171.0


In [17]:
coordinates ={
    'tattapani': [31.2488, 77.0895],
    'shimla': [31.1050, 77.164],
    'waknaghat': [31.0079, 77.0881],
    'solan': [30.9084, 77.0999],
    'chandigarh': [30.733, 76.7794]
}

map_center = [(coordinates['tattapani'][0] + coordinates['chandigarh'][0]) / 2, (coordinates['tattapani'][1] + coordinates['chandigarh'][1]) / 2]

mymap= folium.Map(location=map_center, zoom_start=5)

#markers
for location, coord in coordinates.items():
    folium.Marker(location=coord, popup=location).add_to(mymap)

path_coordinates = [coordinates[node] for node in shortest_path]

folium.PolyLine(locations=path_coordinates, color='blue', weight=2.5, opacity=1).add_to(mymap)

mymap.save('map.html')

mymap
